In [ ]:
import json
import os

from dotenv import load_dotenv

from pydantic import BaseModel, ValidationError, Field
from typing import List

from openai import AsyncOpenAI

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread

from semantic_kernel.functions import KernelArguments

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [ ]:
class SubTask(BaseModel):
    assigned_agent: str = Field(
        description="The specific agent assigned to handle this subtask")
    task_details: str = Field(
        description="Detailed description of what needs to be done for this subtask")


class TravelPlan(BaseModel):
    main_task: str = Field(
        description="The overall travel request from the user")
    subtasks: List[SubTask] = Field(
        description="List of subtasks broken down from the main task, each assigned to a specialized agent")

In [ ]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = """You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general requests"""

# Create the prompt execution settings and configure the Pydantic model response format
settings = OpenAIChatPromptExecutionSettings(response_format=TravelPlan)

agent = ChatCompletionAgent(
    service=chat_completion_service,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings) 
)

In [ ]:
from IPython.display import display, HTML


async def main():
    # Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    # Respond to user input
    user_inputs = [
        "Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne",
    ]

    for user_input in user_inputs:
        
        # Start building HTML output
        html_output = "<div style='margin-bottom:10px'>"
        html_output += "<div style='font-weight:bold'>User:</div>"
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += "</div>"

        # Collect the agent's response
        response = await agent.get_response(messages=user_input, thread=thread)
        thread = response.thread

        try:
            # Try to validate the response as a TravelPlan
            travel_plan = TravelPlan.model_validate(json.loads(response.message.content))

            # Display the validated model as formatted JSON
            formatted_json = travel_plan.model_dump_json(indent=4)
            html_output += "<div style='margin-bottom:20px'>"
            html_output += "<div style='font-weight:bold'>Validated Travel Plan:</div>"
            html_output += f"<pre style='margin-left:20px; padding:10px; border-radius:5px;'>{formatted_json}</pre>"
            html_output += "</div>"
        except ValidationError as e:
            # Handle validation errors
            html_output += "<div style='margin-bottom:20px; color:red;'>"
            html_output += "<div style='font-weight:bold'>Validation Error:</div>"
            html_output += f"<pre style='margin-left:20px;'>{str(e)}</pre>"
            html_output += "</div>"
            # Add this to see what the response contains for debugging
            html_output += "<div style='margin-bottom:20px;'>"
            html_output += "<div style='font-weight:bold'>Raw Response:</div>"
            html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.content}</div>"
            html_output += "</div>"

        html_output += "<hr>"

        # Display formatted HTML
        display(HTML(html_output))

await main()

# Úvod

Tento dokument poskytuje přehled o tom, jak používat @@INLINE_CODE_0@@ k dosažení optimálních výsledků.

## Požadavky

Než začnete, ujistěte se, že máte následující:

- Nainstalovaný @@INLINE_CODE_1@@
- Přístup k @@INLINE_CODE_2@@
- Základní znalosti o @@INLINE_CODE_3@@

## Instalace

Chcete-li nainstalovat @@INLINE_CODE_4@@, postupujte podle těchto kroků:

1. Otevřete terminál.
2. Zadejte následující příkaz:

   ```json
User:
Create a travel plan for a family of 4, with 2 kids, from Singapore to Melboune
Validated Travel Plan:
{
    "main_task": "Plan a family trip from Singapore to Melbourne for 4 people including 2 kids.",
    "subtasks": [
        {
            "assigned_agent": "FlightBooking",
            "task_details": "Book round-trip flights from Singapore to Melbourne for 2 adults and 2 children."
        },
        {
            "assigned_agent": "HotelBooking",
            "task_details": "Find and book a family-friendly hotel in Melbourne that accommodates 4 people."
        },
        {
            "assigned_agent": "CarRental",
            "task_details": "Arrange for a car rental in Melbourne suitable for a family of 4."
        },
        {
            "assigned_agent": "ActivitiesBooking",
            "task_details": "Plan and book family-friendly activities in Melbourne suitable for kids."
        },
        {
            "assigned_agent": "DestinationInfo",
            "task_details": "Provide information about Melbourne, including attractions, dining options, and family-oriented activities."
        }
    ]
}
```

3. Po dokončení instalace ověřte, že @@INLINE_CODE_4@@ funguje správně.

## Použití

### Základní příklad

Následující příklad ukazuje, jak použít @@INLINE_CODE_5@@ k vytvoření jednoduchého výstupu:

@@CODE_BLOCK_1@@

### Pokročilé funkce

Pokud chcete využít pokročilé funkce, podívejte se na následující příklad:

@@CODE_BLOCK_2@@

## Časté problémy

### Problém: @@INLINE_CODE_6@@ nefunguje

**Možné příčiny:**

- @@INLINE_CODE_7@@ není správně nakonfigurován.
- Chybí požadované závislosti.

**Řešení:**

1. Zkontrolujte konfiguraci pomocí @@INLINE_CODE_8@@.
2. Ujistěte se, že všechny závislosti jsou nainstalovány.

### Problém: Chyba při spuštění @@INLINE_CODE_9@@

**Možné příčiny:**

- Konflikt verzí.
- Nesprávné parametry.

**Řešení:**

1. Aktualizujte @@INLINE_CODE_10@@ na nejnovější verzi.
2. Zkontrolujte parametry v @@INLINE_CODE_11@@.

## Závěr

Dodržováním těchto pokynů byste měli být schopni úspěšně používat @@INLINE_CODE_12@@. Pokud narazíte na další problémy, podívejte se na oficiální dokumentaci na @@INLINE_CODE_13@@.



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o přesnost, mějte na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
